In [1]:
import tensorflow as tf
import edward as ed

## Primitive stochastic functions

* Use **Edward**'s models
* Can create custom distributions using transforms or by inheriting directly from `RandomVariable`

### Drawing samples from unit Normal

In [2]:
loc = 0.   # mean zero
scale = 1. # unit variance

In [3]:
normal = ed.models.Normal(loc, scale) # creates a normal distribution object

In [4]:
x = normal.sample() # draws a sample from N(0, 1)

with tf.Session() as sess:
    vx, vlp = sess.run([x, normal.log_prob(x)])
    print("sample", vx)
    print("log prob", vlp)

sample 1.0821803
log prob -1.5044956


### Sampling

* Samples are named hierarchically

In [5]:
x = normal.sample(name="x")

with tf.Session() as sess:
    vx = sess.run(x)
    print("sample", x, vx)

sample Tensor("Normal/x/Reshape:0", shape=(), dtype=float32) -0.9265542


## Simple model

In [6]:
def weather():
    cloudy = ed.models.Bernoulli(0.3).sample(name="cloudy")
    cloudy = tf.cond(tf.equal(cloudy, 1), lambda: tf.constant('cloudy'), lambda: tf.constant('sunny'))
    mean_temp = tf.cond(tf.equal(cloudy, 'cloudy'), lambda: tf.constant(55.0), lambda: tf.constant(75.0))
    scale_temp = tf.cond(tf.equal(cloudy, 'cloudy'), lambda: tf.constant(10.0), lambda: tf.constant(15.0))
    temp = ed.models.Normal(mean_temp, scale_temp).sample(name='temp')
    return cloudy, temp

with tf.Session() as sess:
    (cloudy, temp) = weather()
    for _ in range(3):
        print(sess.run((cloudy, temp)))

(b'sunny', 45.704914)
(b'sunny', 71.09928)
(b'cloudy', 67.831314)


In [7]:
def ice_cream_sales():
    cloudy, temp = weather()
    expected_sales = tf.cond(tf.equal(cloudy, "sunny"), lambda: tf.constant(200.0), lambda: tf.constant(50.0))
    ice_cream = ed.models.Normal(expected_sales, 10.0).sample(name='ice_scream')
    return ice_cream

with tf.Session() as sess:
    print(sess.run(ice_cream_sales()))

66.29641


## Stochastic Recursion, Higher-order Stochastic Functions, and Random Control Flow 

* **Edward** builds on **TensorFlow** and thus does not support general recursion.
* Instead, it supports a restricted form of `while` loops (through `tf.while_loop`).
* However, many recursive functions can still be converted into usages of `tf.while_loop`.

In [8]:
def geometric(p):
    return tf.while_loop(
        cond=lambda _: tf.equal(ed.models.Bernoulli(probs=p).sample(), 0), # name is automatically generated
        body=lambda t: tf.add(t, 1),
        loop_vars=[tf.constant(0)],
    )

with tf.Session() as sess:
    acc = sess.run(geometric(0.001))
    print(acc)

177


## Inference

In [5]:
def scale(guess):
    var = 1.0
    # The prior over weight encodes our uncertainty about our guess
    weight = ed.models.Normal(guess, var)
    # This encodes our belief about the noisiness of the scale:
    # the measurement fluctuates around the true weight
    measurement = ed.models.Normal(weight, 0.75)
    return weight, measurement

weight, measurement = scale(8.5)
posterior = ed.models.Empirical(tf.Variable(tf.zeros(10000)))
inference = ed.HMC({weight: posterior}, {measurement: 9.5})
inference.run()
vmean, vstd = ed.get_session().run((posterior.mean(), posterior.stddev()))
print("mean", vmean)
print("stddev", vstd)

10000/10000 [100%] ██████████████████████████████ Elapsed: 8s | Acceptance Rate: 0.990
mean 9.133204
stddev 0.60152006


## Variational Inference

In [6]:
def guide(guess):
    qalpha = tf.Variable(ed.models.Uniform(0.0, 1.0).sample() + guess, name='a')
    qbeta = tf.Variable(ed.models.Uniform(0.0, 1.0).sample(), name='b')
    qweight = ed.models.Normal(qalpha, qbeta)
    return qalpha, qbeta, qweight

guess = tf.Variable(8.5)
weight, measurement = scale(guess)
qalpha, qbeta, qweight = guide(guess)
inference = ed.KLqp({weight: qweight}, {measurement: 9.5})
inference.run(n_samples=10, n_iter=10000)
vmean, vstd = ed.get_session().run((qalpha, qbeta))
print("mean", vmean)
print("stddev", vstd)

10000/10000 [100%] ██████████████████████████████ Elapsed: 9s | Loss: 0.975
mean 9.500995
stddev 0.5999705


## Variational Auto Encoder